In [1]:
import numpy as np
import pandas as pd


In [20]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [3]:
df = pd.read_csv('covid_toy.csv')

In [5]:
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [7]:
df['cough'].value_counts()

cough
Mild      62
Strong    38
Name: count, dtype: int64

In [13]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [15]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(df.drop(columns=['has_covid']),df['has_covid'],test_size=0.2)

In [17]:
x_train

,age,gender,fever,cough,city
25,23,Male,NaN,Mild,Mumbai
56,71,Male,NaN,Strong,Kolkata
58,23,Male,98.0,Strong,Mumbai
42,27,Male,100.0,Mild,Delhi
15,70,Male,103.0,Strong,Kolkata
...,...,...,...,...,...
93,27,Male,100.0,Mild,Kolkata
32,34,Female,101.0,Strong,Delhi
49,44,Male,104.0,Mild,Mumbai
43,22,Female,99.0,Mild,Bangalore


## Without column Transformer (Aam Zindagi)

In [39]:
# adding simple imputer to fever col
si = SimpleImputer()
x_train_fever = si.fit_transform(x_train[['fever']])  # missing values get imputer with column mean

# also the test data
x_test_fever = si.fit_transform(x_test[['fever']])
x_train_fever.shape


(80, 1)

In [45]:
# ordinal encoding --> cough
oe = OrdinalEncoder(categories=[['Mild','Strong']])
x_train_cough = oe.fit_transform(x_train[['cough']])

# also the test data
x_test_cough = oe.fit_transform(x_test[['cough']])
x_train_cough.shape

(80, 1)

In [53]:
# OneHot Encoding --> gender ,city
ohe = OneHotEncoder(drop='first',sparse=False,dtype=int)
x_train_gender_city = ohe.fit_transform(x_train[['gender','city']])

# also the test data
x_test_gender_city = ohe.fit_transform(x_test[['gender','city']])
x_train_gender_city.shape

C:\Users\Khan2\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\Khan2\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


(80, 4)

In [81]:
# Extracting Age
x_train_age = x_train.drop(columns=['gender','fever','cough','city']).values

# also the test data
x_test_age = x_test.drop(columns=['gender','fever','cough','city']).values
x_train_age.shape,x_test_age.shape

((80, 1), (20, 1))

In [87]:
x_train_transformed = np.concatenate((x_train_age,x_train_fever,x_train_gender_city,x_train_cough),axis = 1)
# also the test data
x_test_transformed = np.concatenate((x_test_age,x_test_fever,x_test_gender_city,x_test_cough),axis=1)
x_train_transformed.shape


(80, 7)

## using column tranformer mentos zindagi

In [90]:
from sklearn.compose import ColumnTransformer

In [94]:
transformer = ColumnTransformer(transformers=[('tf1',SimpleImputer(),['fever']),('tf2',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
                                            ('tf3',OneHotEncoder(drop='first',dtype=int,sparse=False),['gender','city'])],remainder='passthrough')

In [98]:
transformer.fit_transform(x_train).shape

C:\Users\Khan2\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


(80, 7)